In [12]:
%load_ext autoreload
%autoreload
%matplotlib inline

import pandas as pd
import numpy as np
import qgrid
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, load_csv, write_xlsx, load_xlsx
from pa_lib.data import (
    calc_col_partitioned,
    clean_up_categoricals,
    unfactorize,
    flatten,
    replace_col,
    cond_col,
    desc_col,
    unfactorize,
    as_dtype,
    flatten_multi_index_cols,
)
from pa_lib.util import obj_size, cap_words, normalize_rows, clear_row_max
from pa_lib.log import time_log, info
from pa_lib.vis import dive

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load EK data

In [15]:
data_files('*ek*')

,size,mtime
name,,
bd_by_week.feather,24.1 MB,10.07.19 15:56:05
bd_cluster_ek_info.feather,4.1 MB,15.08.19 15:46:00
bd_ek_minmax.feather,1.9 MB,10.07.19 15:34:53
bd_long_by_week.feather,23.3 MB,10.07.19 10:25:41
pv_by_week.feather,11.1 MB,22.05.19 18:01:14


In [16]:
ek_info = load_bin('bd_cluster_ek_info.feather')

2019-08-20 15:56:51 [INFO] Reading from file C:\Users\kpf\data\bd_cluster_ek_info.feather
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
2019-08-20 15:56:51 [INFO] Finished loading binary file in 0.16s (0.05s CPU)


# Prepare PLZ mapping data

In [29]:
data_files('*.xlsx')

,size,mtime
name,,
do-t-09.02-gwr-37.xlsx,414.7 KB,20.08.19 17:00:54
ppi_data.xlsx,74.5 MB,18.07.19 17:13:43
ppi_kamp.xlsx,122.3 KB,31.07.19 15:26:30
ppi_kamp_by_std_publ.xlsx,10.1 KB,18.07.19 17:14:16
ppi_kamp_hilo.xlsx,150.3 KB,18.07.19 17:15:20
ppi_kamp_hilo_by_std_publ.xlsx,18.7 KB,18.07.19 17:15:24
sbbnutz_columns_2014.xlsx,16.0 KB,03.07.19 16:42:07
sbbnutz_data_2014.xlsx,1.6 MB,03.07.19 16:42:07
test.xlsx,9.9 KB,11.07.19 16:57:20


In [47]:
plz = load_xlsx("do-t-09.02-gwr-37.xlsx", sheet_name="PLZ4").rename(
    columns={"PLZ4": "PLZ", "%_IN_GDE": "PRC", "KTKZ": "KANTON", "GDENAMK": "NAME"}
)

2019-08-20 17:11:46 [INFO] Reading from file C:\Users\kpf\data\do-t-09.02-gwr-37.xlsx
2019-08-20 17:11:46 [INFO] Finished loading xlsx file in 0.64s (0.66s CPU)


In [48]:
plz.head(10)

,PLZ,PRC,KANTON,GDENR,NAME
0,8914,100.00,ZH,1,Aeugst am Albis
1,8909,14.34,ZH,2,Affoltern am Albis
2,8910,85.66,ZH,2,Affoltern am Albis
3,8906,100.00,ZH,3,Bonstetten
4,6340,0.77,ZH,4,Hausen am Albis
5,8915,72.06,ZH,4,Hausen am Albis
6,8925,27.17,ZH,4,Hausen am Albis
7,8908,100.00,ZH,5,Hedingen
8,8926,100.00,ZH,6,Kappel am Albis
9,8934,100.00,ZH,7,Knonau


In [40]:
plz.PLZ.value_counts()

1148    8
3053    7
9107    7
2345    7
6110    7
       ..
2843    1
6937    1
6933    1
6929    1
1344    1
Name: PLZ, Length: 3184, dtype: int64

In [54]:
def collect(s, sep=","):
    return sep.join(map(str, s[s.notna()].unique()))


plz.groupby("PLZ").agg({"KANTON": ["nunique", collect]}).sort_values(
    ("KANTON", "nunique"), ascending=False
)

KANTON          
     nunique   collect
PLZ                   
9107       3  AR,AI,SG
2814       3  SO,BL,JU
6390       3  UR,OW,NW
2827       3  BE,SO,JU
9428       3  AR,AI,SG
...      ...       ...
3508       1        BE
3510       1        BE
3512       1        BE
3513       1        BE
9658       1        SG

[3184 rows x 2 columns]

In [62]:
def main_features(df):
    return df.loc[df.PRC.idxmax(), ['GDENR', 'NAME', 'KANTON']]

plz_unique = plz.groupby('PLZ').apply(main_name)

# Load data on economic regions

In [233]:
reg = load_xlsx("Raumgliederungen.xlsx", sheet_name="Daten", skiprows=0, header=1)

# drop unused
reg.drop(0, axis="index", inplace=True)
reg.drop(["Bezirks-nummer", "Kantons-nummer"], axis="columns", inplace=True)

# rename columns
reg.rename(
    columns={
        "BFS Gde-nummer": "GDENR",
        "Gemeindename": "NAME",
        "Kanton": "KANTON",
        "Bezirksname": "BEZIRK",
        "Arbeitsmarktgrossregionen 2018": "GROSSREGION_ID",
        "Arbeitsmarktregionen 2018": "REGION_ID",
    },
    inplace=True,
)

# fix data types
reg = reg.astype({"GDENR": "int64", "GROSSREGION_ID": "int64", "REGION_ID": "int64"})

2019-08-21 17:30:49 [INFO] Reading from file C:\Users\kpf\data\Raumgliederungen.xlsx
2019-08-21 17:30:49 [INFO] Finished loading xlsx file in 0.2s (0.19s CPU)


In [132]:
reg.head()

,GDENR,NAME,KANTON,BEZIRK,GROSSREGION_ID,REGION_ID
1,1,Aeugst am Albis,ZH,Affoltern,12,12031
2,2,Affoltern am Albis,ZH,Affoltern,12,12031
3,3,Bonstetten,ZH,Affoltern,12,12034
4,4,Hausen am Albis,ZH,Affoltern,11,11060
5,5,Hedingen,ZH,Affoltern,12,12031


### Grossregionen

In [133]:
reg_grossreg = load_xlsx(
    "Raumgliederungen.xlsx",
    sheet_name="CH1+CL_GBAE2018+1.0",
    skiprows=0,
    header=1,
    index_col=0,
)

2019-08-21 12:54:18 [INFO] Reading from file C:\Users\kpf\data\Raumgliederungen.xlsx
2019-08-21 12:54:19 [INFO] Finished loading xlsx file in 0.17s (0.17s CPU)


In [134]:
reg_grossreg.head()

,Label
Code,
1,Region Genf
2,Region Lausanne
3,Region Neuenburg
4,Region Freiburg
5,Region Biel–Jura


### Regionen

In [135]:
reg_reg = load_xlsx(
    "Raumgliederungen.xlsx",
    sheet_name="CH1+CL_BAE2018+1.0",
    skiprows=0,
    header=1,
    index_col=0,
)

2019-08-21 12:54:25 [INFO] Reading from file C:\Users\kpf\data\Raumgliederungen.xlsx
2019-08-21 12:54:25 [INFO] Finished loading xlsx file in 0.17s (0.17s CPU)


In [136]:
reg_reg.head()

,Label
Code,
1011,Vernier–Lancy
1012,Genève
1013,Le Grand-Saconnex
1014,Nyon
1015,Thônex–Chêne-Bougeries


### Merge Grossregionen and Regionen

In [234]:
reg = (
    reg.assign(
        REG=reg_reg.loc[reg.REGION_ID].values,
        GROSSREG=reg_grossreg.loc[reg.GROSSREGION_ID].values,
    )
    .drop(["GROSSREGION_ID", "REGION_ID"], axis="columns")
    .set_index("GDENR")
)

In [141]:
reg.head()

,NAME,KANTON,BEZIRK,GROSSREG,REG
GDENR,,,,,
1,Aeugst am Albis,ZH,Affoltern,Region Zürich,Dietikon–Schlieren
2,Affoltern am Albis,ZH,Affoltern,Region Zürich,Dietikon–Schlieren
3,Bonstetten,ZH,Affoltern,Region Zürich,Horgen–Wädenswil
4,Hausen am Albis,ZH,Affoltern,Zentralschweiz,Zug
5,Hedingen,ZH,Affoltern,Region Zürich,Dietikon–Schlieren


# Add region info to customers

### Find a GDENR for each swiss customer

In [251]:
swiss = ek_info.EK_Land == "SCHWEIZ"

In [252]:
ek_info["GDENR_PLZ"] = plz_unique.reindex(
    pd.to_numeric(ek_info.EK_Plz, errors="coerce").fillna(-1).astype("int64")
).GDENR.values

In [253]:
gdenr_by_name = plz.groupby('NAME').agg({'GDENR': 'first'})

ek_info["GDENR_NAME"] = (
    gdenr_by_name.reindex(ek_info.EK_Ort).values
)

In [254]:
def firstnames(s):
    s = s.fillna("")
    return s.str.partition(expand=False).apply(lambda x: x[0])


gdenr_by_firstname = (
    plz.assign(FIRSTNAME=firstnames(plz.NAME))
    .groupby("FIRSTNAME")
    .agg({"GDENR": "first"})
)

ek_info["GDENR_FIRSTNAME"] = gdenr_by_firstname.reindex(
    firstnames(ek_info.EK_Ort)
).values

In [255]:
ek_info["GDENR_SPECIAL"] = gdenr_by_name.reindex(
    ek_info.EK_Ort.replace(
        {
            "Schönbühl Einkaufszentrum": "Bern",
            "Emmenbrücke 1": "Emmen",
            "Glattzentrum b. Wallisellen": "Wallisellen",
            "Zürich-Flughafen": "Kloten",
            "Büsingen": "Schaffhausen",
            "Serfontana": "Chiasso",
            "Triesen": "Sevelen",
            "Campione d'Italia": "Bissone",
        }
    )
).values

In [256]:
ek_info["GDENR"] = (
    ek_info.GDENR_PLZ
    .fillna(ek_info.GDENR_NAME)
    .fillna(ek_info.GDENR_FIRSTNAME)
    .fillna(ek_info.GDENR_SPECIAL)
)

# Fix mismatches of non-swiss customers
ek_info.loc[~swiss, 'GDENR'] = None

ek_info.drop(
    "GDENR_PLZ GDENR_NAME GDENR_FIRSTNAME GDENR_SPECIAL".split(), axis="columns", inplace=True
)

#### Sanity check: this should be empty!

In [257]:
not_matched = ek_info.loc[
    ek_info.GDENR.isnull()
    & (ek_info.EK_Land == "SCHWEIZ"),
    ["EK_Plz", "EK_Ort"],
]

not_matched

,EK_Plz,EK_Ort


### Join region info on GDENR

In [263]:
ek_info = ek_info.merge(reg, on='GDENR', how='left')

In [267]:
ek_info.head(20)

,Endkunde_NR,Endkunde,EK_Aktiv,EK_Land,EK_Plz,EK_Ort,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Kamp_Erfass_Jahr_min,Kamp_Erfass_Jahr_max,GDENR,NAME,KANTON,BEZIRK,REG,GROSSREG
0,100034,Maurer + Salzmann AG,1,SCHWEIZ,8408,Winterthur,ACE 2 ACE outdoor media ag,WG - Dienstleistung,720,"720,405,400,295",4,2009,2018,230.0,Winterthur,ZH,Winterthur,Winterthur,Region Zürich
1,100039,Bank BSU Genossenschaft,1,SCHWEIZ,8610,Uster,"walder, werber werbeagentur ag",WG - Finanzwirtschaft,470,"470,720",2,2009,2019,198.0,Uster,ZH,Uster,Uster–Dübendorf,Region Zürich
2,100061,Mundwiler Juwelen AG,1,SCHWEIZ,8400,Winterthur,None,WG - Persönlicher Bedarf,555,555,1,2009,2009,230.0,Winterthur,ZH,Winterthur,Winterthur,Region Zürich
3,100064,Senn Communication,1,SCHWEIZ,8610,Uster,None,WG - Dienstleistung,720,"901,960",2,2009,2017,198.0,Uster,ZH,Uster,Uster–Dübendorf,Region Zürich
4,100066,"walder, werber werbeagentur ag",1,SCHWEIZ,8610,Uster,"walder, werber werbeagentur ag",WG - Dienstleistung,720,"780,720",2,2013,2019,198.0,Uster,ZH,Uster,Uster–Dübendorf,Region Zürich
5,100083,Goodyear Dunlop Tires Suisse SA,1,SCHWEIZ,8604,Volketswil,a.ha GmbH,WG - Fahrzeugzubehör,390,390,1,2011,2019,199.0,Volketswil,ZH,Uster,Uster–Dübendorf,Region Zürich
6,100092,Navyboot AG,1,SCHWEIZ,8001,Zürich,Kinetic Worldwide Switzerland AG,WG - Bekleidung / Wäsche,195,195,1,2009,2015,261.0,Zürich,ZH,Zürich,Zürich,Region Zürich
7,100095,Zomag AG,1,SCHWEIZ,8620,Wetzikon ZH,ACE 2 ACE outdoor media ag,WG - Dienstleistung,720,720,1,2009,2019,121.0,Wetzikon (ZH),ZH,Hinwil,Wetzikon (ZH),Region Zürich
8,100097,Biomed AG,1,SCHWEIZ,8600,Dübendorf,mediaschneider ag,WG - Pharma Produkte,366,"366,800,720",3,2009,2018,191.0,Dübendorf,ZH,Uster,Uster–Dübendorf,Region Zürich
9,100098,Maison Astor Coiffure GmbH,1,SCHWEIZ,8304,Wallisellen,None,WG - Haarpflege,332,"334,332",2,2008,2018,69.0,Wallisellen,ZH,Bülach,Uster–Dübendorf,Region Zürich


#### Sanity check: this should be empty!

In [266]:
ek_info.loc[(ek_info.EK_Land != 'SCHWEIZ') & ek_info.KANTON.notnull()]

,Endkunde_NR,Endkunde,EK_Aktiv,EK_Land,EK_Plz,EK_Ort,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Kamp_Erfass_Jahr_min,Kamp_Erfass_Jahr_max,GDENR,NAME,KANTON,BEZIRK,REG,GROSSREG
